In [1]:
import os
import tqdm as tqdm
import numpy as np
import copy

# 处理服务器中evo的可视化问题
import evo
from evo.tools.settings import SETTINGS
SETTINGS['plot_backend'] = 'Agg'

from evo.tools import file_interface, plot
from evo.core.geometry import GeometryException
import evo.main_ape as main_ape
from evo.core import sync, metrics
from evo.core.trajectory import PoseTrajectory3D

import matplotlib.pyplot as plt #绘图

print("Successfully import ultils")

Successfully import ultils


In [2]:
def make_evo_traj_gt(poses_N_x_7, tss_us):
    assert poses_N_x_7.shape[1] == 7
    assert poses_N_x_7.shape[0] > 10
    assert tss_us.shape[0] == poses_N_x_7.shape[0]

    traj_evo = PoseTrajectory3D(
        positions_xyz=poses_N_x_7[:,:3],
        # orientations_quat_wxyz=poses_N_x_7[:,3:],
        orientations_quat_wxyz = poses_N_x_7[:, [6,3,4,5]],#gt存储的是xyzw
        timestamps=tss_us/1e6)#转换为秒
    return traj_evo

def make_evo_traj_deio(poses_N_x_7, tss_us):
    assert poses_N_x_7.shape[1] == 7
    assert poses_N_x_7.shape[0] > 10
    assert tss_us.shape[0] == poses_N_x_7.shape[0]

    traj_evo = PoseTrajectory3D(
        positions_xyz=poses_N_x_7[:,:3],
        # orientations_quat_wxyz=poses_N_x_7[:,3:],
        orientations_quat_wxyz = poses_N_x_7[:, [5,6,3,4]],#存储的是yzwx(由于原代码的bug导致的,注意统一代码输出为xyzw而evo中需要用的是wxyz即可)
        timestamps=tss_us/1e6)#转换为秒
    return traj_evo

def load_gt_us(path, skiprows=0):
    traj_ref = np.loadtxt(path, delimiter=" ", skiprows=skiprows)
    tss_gt_us = traj_ref[:, 0].copy() 
    assert np.all(tss_gt_us == sorted(tss_gt_us))
    assert traj_ref.shape[0] > 0
    assert traj_ref.shape[1] == 8

    return tss_gt_us, traj_ref[:, 1:]

def plot_trajectory_inxyplane(pred_traj, gt_traj, align=True, _n_to_align=-1,correct_scale=True, max_diff_sec=1.0,title="", filename=""):
    #将两个轨迹对齐(时间维度上的)
    gt_traj, pred_traj = sync.associate_trajectories(gt_traj, pred_traj, max_diff=max_diff_sec)

    # 对齐轨迹(空间维度上的)
    if align:
        try:
            pred_traj.align(gt_traj, correct_scale=correct_scale,n=_n_to_align)
        except GeometryException as e:
            print("Plotting error:", e)

    plot_collection = plot.PlotCollection("PlotCol")

    fig = plt.figure(figsize=(8, 8))
    plot_mode = plot.PlotMode.xy
    ax = plot.prepare_axis(fig, plot_mode)
    ax.set_title(title)
    if gt_traj is not None:
        plot.traj(ax, plot_mode, gt_traj, '--', 'gray', "Ground Truth")
    plot.traj(ax, plot_mode, pred_traj, '-', 'blue', "Predicted")
    
    plot_collection.add_figure("traj (error)", fig)

    plt.show()

print("Successfully define several functions")

Successfully define several functions


In [3]:
print("Evaluation for EDS dataset")
indir="/media/lfl-data2/EDS/" # your dataset path
cur_dir=os.getcwd() #current file directoin

target_dirs = {
                "00_peanuts_dark",
                "01_peanuts_light",
                "02_rocket_earth_light",
                "03_rocket_earth_dark",
                "06_ziggy_and_fuzz",
                "07_ziggy_and_fuzz_hdr",
                "08_peanuts_running",
                "09_ziggy_flying_pieces",
                "11_all_characters",
                }

for root, dirs, files in os.walk(indir):
    for d in dirs:
        # 构建完整路径 data_path
        datapath_val = os.path.join(root, d)

        # 检查是否为目标文件夹之一
        if os.path.basename(datapath_val) in target_dirs:
            sequence_name = os.path.basename(datapath_val)

            # 获取轨迹
            tss_traj_s, traj_gt = load_gt_us(os.path.join(datapath_val, "stamped_groundtruth.txt"))#获取真实轨迹
            tss_gt_us=tss_traj_s * 1e6#转换为微秒

            # 获取deio估算的轨迹
            if sequence_name == "00_peanuts_dark":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./EDS/2024-11-17_00_peanuts_dark/00_Peanuts_Dark_Trial01.txt"))
            elif sequence_name == "01_peanuts_light":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./EDS/2024-11-14_01_peanuts_light/01_Peanuts_Light_Trial01.txt"))
            elif sequence_name == "02_rocket_earth_light":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./EDS/2024-11-17_02_rocket_earth_light/02_Rocket_Earth_Light_Trial01.txt"))
            elif sequence_name == "03_rocket_earth_dark":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./EDS/2024-11-17_03_rocket_earth_dark/03_Rocket_Earth_Dark_Trial01.txt"))
            elif sequence_name == "06_ziggy_and_fuzz":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./EDS/2024-11-17_06_ziggy_and_fuzz/06_Ziggy_And_Fuzz_Trial01.txt"))
            elif sequence_name == "07_ziggy_and_fuzz_hdr":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./EDS/2024-11-19_07_ziggy_and_fuzz_hdr/07_Ziggy_And_Fuzz_Hdr_Trial01.txt"))
            elif sequence_name == "08_peanuts_running":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./EDS/2024-11-17_08_peanuts_running/08_Peanuts_Running_Trial01.txt"))
            elif sequence_name == "09_ziggy_flying_pieces":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./EDS/2024-11-14_09_ziggy_flying_pieces/09_Ziggy_Flying_Pieces_Trial01.txt"))
            elif sequence_name == "11_all_characters":
                tss_deio_us, traj_deio = load_gt_us(os.path.join(cur_dir, "./EDS/2024-11-17_11_all_characters/11_All_Characters_Trial01.txt"))                 

            evoGT = make_evo_traj_gt(traj_gt, tss_gt_us)
            evoEst = make_evo_traj_deio(traj_deio, tss_deio_us)
            gtlentraj = evoGT.get_infos()["path length (m)"]#获取轨迹长度
            evoGT, evoEst = sync.associate_trajectories(evoGT, evoEst, max_diff=1)
            _n_to_align=-1;
            ape_trans = main_ape.ape(copy.deepcopy(evoGT), copy.deepcopy(evoEst), pose_relation=metrics.PoseRelation.translation_part, align=True,n_to_align=_n_to_align, correct_scale=True)

            # print(f"\033[31m EVO结果：{ape_trans}\033[0m");
            MPE = ape_trans.stats["mean"] / gtlentraj * 100
            # print(f"MPE is {MPE:.02f}") #注意只保留两位小数
            evoATE = ape_trans.stats["rmse"]*100

            res_str = f"\nATE[cm]: {evoATE:.02f} | MPE[%/m]: {MPE:.02f}"
            
            print(f"{sequence_name}: {res_str}")
            
    # 使用break限制os.walk只遍历indir的第一层
    break

Evaluation for EDS dataset
00_peanuts_dark: 
ATE[cm]: 1.77 | MPE[%/m]: 0.15
01_peanuts_light: 
ATE[cm]: 16.27 | MPE[%/m]: 0.69
02_rocket_earth_light: 
ATE[cm]: 15.41 | MPE[%/m]: 0.84
03_rocket_earth_dark: 
ATE[cm]: 8.91 | MPE[%/m]: 0.34
06_ziggy_and_fuzz: 
ATE[cm]: 10.39 | MPE[%/m]: 0.15
07_ziggy_and_fuzz_hdr: 
ATE[cm]: 23.82 | MPE[%/m]: 0.39
08_peanuts_running: 
ATE[cm]: 19.96 | MPE[%/m]: 0.22
09_ziggy_flying_pieces: 
ATE[cm]: 3.84 | MPE[%/m]: 0.26
11_all_characters: 
ATE[cm]: 31.55 | MPE[%/m]: 0.42
